In [1]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [3]:
whos

Variable   Type      Data/Info
------------------------------
os         module    <module 'os' from '/data/<...>da3/lib/python3.8/os.py'>


In [4]:
!free -g

              total        used        free      shared  buff/cache   available
Mem:            187           2         182           0           1         182
Swap:            11           0          11


In [5]:
!nvidia-smi

Fri Oct  8 09:22:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:18:00.0 Off |                    0 |
| N/A   35C    P0    44W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [6]:
# import os
# from datetime import datetime; now = datetime.now

# analysis_name = 'CVAE_'+str(now())
# save_dir = os.path.join('../Assets/tf_weights',analysis_name)
# if not os.path.exists(save_dir):
#     os.mkdir(save_dir)
    
# print(analysis_name)

In [7]:
# CHECK GPU
import nvidia_smi
try:
    nvidia_smi.nvmlInit()

    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
    # card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

    print("Total memory:", (info.total/1024/1024/1024))
    print("Free memory:", (info.free/1024/1024/1024))
    print("Used memory:", (info.used/1024/1024/1024))

    nvidia_smi.nvmlShutdown()
except:
    pass

Total memory: 15.78173828125
Free memory: 15.7816162109375
Used memory: 0.0001220703125


In [8]:
## SET UP

In [9]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt

from functools import partial
from tqdm import tqdm
from umap import UMAP
tqdm = partial(tqdm, position=0, leave=True) 

In [10]:
df = pd.read_csv('../Data/ABIDE_df2.csv')
df = df.iloc[~pd.isna(df['bids_folder']).values]
data_dir = '../Assets/fc_mats_32smooth_new/'

In [11]:
files = [file for file in os.listdir(data_dir) if file.endswith('.npy')]
files.sort()
files[0:5]

['sub-CMUa0050642.npy',
 'sub-CMUa0050646.npy',
 'sub-CMUa0050647.npy',
 'sub-CMUa0050649.npy',
 'sub-CMUa0050653.npy']

In [12]:
subs_with_matFiles = [file.split('.')[0] for file in files]
df = df.iloc[np.array([sub in subs_with_matFiles for sub in df['bids_folder'].values])]
n_subs = len(df)
print(n_subs)

# Check if subject has .nii file
has_file = np.array([os.path.exists(os.path.join(data_dir,sub + '.npy')) for sub in df['bids_folder'].values])
print(has_file.mean())
assert has_file.mean()==1.0, 'fuck'

assert len(subs_with_matFiles)==len(df),'no'
df = df.sort_values(by='bids_folder')
assert all([subs_with_matFiles[i]==df['bids_folder'].values[i] for i in range(n_subs)]), 'out of order'

df.index = np.arange(n_subs)

1048
1.0


In [30]:
%%time
from importlib import reload
import helper_funcs;reload(helper_funcs);from helper_funcs import *
del helper_funcs
import make_models;reload(make_models);from make_models import *
del make_models

from IPython import display
import sys
from sklearn.decomposition import PCA
import seaborn as sns

import tensorflow as tf
reload(tf)

CPU times: user 2.69 ms, sys: 950 µs, total: 3.64 ms
Wall time: 6.36 ms


<module 'tensorflow' from '/data/aglinska/anaconda3/lib/python3.8/site-packages/tensorflow/__init__.py'>

In [14]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [15]:
data_dir = '../Assets/fc_mats_32smooth_new'
data_loader = cvae_data_loader(data_dir=data_dir, df=df, batch_size=32)
batch_asd,batch_td,batch_df = data_loader.get_batch()

In [16]:
data_size = np.hstack((n_subs,batch_asd.shape[1:]))

In [17]:
import make_models;reload(make_models);from make_models import *
batch_size = 16

cvae, z_encoder, s_encoder, cvae_decoder = get_fMRI_CVAE_4D(input_shape=tuple(data_size[1::]),
                                                             latent_dim=32,
                                                             beta=1,
                                                             gamma=100,
                                                             disentangle=True,
                                                             bias=True,
                                                             batch_size = batch_size,
                                                             kernel_size = 3,
                                                             filters = 8,
                                                             intermediate_dim = 128,
                                                             nlayers = 5,
                                                             learning_rate=0.001,
                                                             opt=None)

In [18]:
tuple(data_size)

(1048, 32, 32, 32, 51)

In [19]:
loss = list()
c_sim = list()
all_rsas = list()

In [20]:
data_loader = cvae_data_loader(data_dir=data_dir, df=df, batch_size=batch_size)

In [110]:
import make_models;reload(make_models);from make_models import *
import helper_funcs;reload(helper_funcs);from helper_funcs import *

for epoch in tqdm(range(100),position=0, leave=True):
    for i in range(data_loader.n_batches):
        
        patient_batch,control_batch,batch_df = data_loader.get_batch() # Get a batch
        hist = cvae.train_on_batch([patient_batch,control_batch]) # pass a batch
        assert not np.isnan(hist),'loss is NaN - you f**cked up'  # check nothing crashed
        loss.append(hist) # keep track of loss
        
        cvae.save_weights(os.path.join(save_dir,'cvae_weights')) # SAVE WEIGHTS
        np.save(os.path.join(save_dir,'cvae_loss.npy'),np.array(loss)) # Save loss
        
        if np.mod(i,10)==0:
            batch_rsas = get_batch_rsas(data_dir,df,z_encoder,s_encoder,batch_size=(df['DX_GROUP'].values==1).sum())
        
        c_sim,all_rsas = cvae_dashboard(data_loader.epoch,
                                        data_loader.b,
                                        i,
                                        cvae,z_encoder,
                                        s_encoder,
                                        loss,
                                        all_rsas,
                                        c_sim,
                                        df,
                                        patient_batch,
                                        control_batch,
                                        batch_rsas,
                                        red='UMAP') # plot training progress
        

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: cannot unpack non-iterable NoneType object

In [93]:
class cvae_data_loader():
    ''' this is the info'''
    def __init__(self,data_dir,df,batch_size=32):
        #self.data = data
        #self.n = data.shape[0]
        self.n = len(df)
        self.data_dir = data_dir
        self.epoch = -1
        self.batch_size = batch_size
        self.df = df
        
        self.new_epoch()
        self.n_batches = int(np.floor(min((len(self.asd_idxs),len(self.td_idxs)))/self.batch_size))
        
    def new_epoch(self):
        #self.df = self.df.iloc[np.random.permutation(np.arange(self.n))]
        #self.df.index = np.arange(self.n)
        self.asd_idxs = np.nonzero((self.df['DX_GROUP'].values==1))[0]
        self.td_idxs = np.nonzero((self.df['DX_GROUP'].values==2))[0]
        
        self.asd_idxs = np.random.permutation(self.asd_idxs)
        self.td_idxs = np.random.permutation(self.td_idxs)
        
        assert all(self.df.iloc[self.asd_idxs]['DX_GROUP'].values==1),'fuck'
        assert all(self.df.iloc[self.td_idxs]['DX_GROUP'].values==2),'fuck'
        
        self.epoch += 1
        self.b = 0
        
        
    def get_batch(self):
        self.b += 1
        
        if self.b==self.n_batches:
            self.new_epoch()
        
        self.batch_asd_idx = self.asd_idxs[np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)]
        self.batch_td_idx = self.td_idxs[np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)]
        
        assert all(self.df['DX_GROUP'].values[self.batch_asd_idx]==1), 'error'
        assert all(self.df['DX_GROUP'].values[self.batch_td_idx]==2), 'error'
        
        self.batch_asd = np.array([np.load(os.path.join(self.data_dir,val+'.npy')) for val in self.df['bids_folder'].values[self.batch_asd_idx]])
        self.batch_td = np.array([np.load(os.path.join(self.data_dir,val+'.npy')) for val in self.df['bids_folder'].values[self.batch_td_idx]])
        self.batch_df = self.df.iloc[np.hstack((self.batch_asd_idx,self.batch_td_idx)),:]
        
        self.batch_asd = self.batch_asd/2
        self.batch_td = self.batch_td/2
        
        assert not (self.batch_asd.max() < 0 or self.batch_asd.max() > 1),'fuck'
        assert not (self.batch_td.max() < 0 or self.batch_td.max() > 1),'fuck'
        
    
        return self.batch_asd,self.batch_td,self.batch_df


In [94]:
batch_size = 16
data_dir = '../Assets/fc_mats_32smooth_new'
data_loader = cvae_data_loader(data_dir=data_dir, df=df, batch_size=batch_size)

In [118]:
batch_asd,batch_td,batch_df = data_loader.get_batch()